# Module 7 - Programming Assignment



In [1]:
import pprint
import copy
pp = pprint.PrettyPrinter(indent=4)

# Forward Planner

## Unify

Use the accompanying `unification.py` file for unification. For this assignment, you're almost certainly going to want to be able to:

1. specify the problem in terms of S-expressions.
2. parse them.
3. work with the parsed versions.

`parse` and `unification` work exactly like the programming assignment for last time.

In [2]:
from unification import parse, unification, is_variable

## Forward Planner

In this assigment, you're going to implement a Forward Planner. What does that mean? If you look in your book, you will not find pseudocode for a forward planner. It just says "use state space search" but this is less than helpful and it's a bit more complicated than that. **(but please please do not try to implement STRIPS or GraphPlan...that is wrong).**

At a high level, a forward planner takes the current state of the world $S_0$ and attempts to derive a plan, basically by Depth First Search. We have all the ingredients we said we would need in Module 1: states, actions, a transition function and a goal test. We have a set of predicates that describe a state (and therefore all possible states), we have actions and we have, at least, an implicit transition function: applying an action in a state causes the state to change as described by the add and delete lists.

Let's say we have a drill that's an item, two places such as home and store, and we know that I'm at home and the drill is at the store and I want to go buy a drill (have it be at home). We might represent that as:

<code>
start_state = [
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Store)"
]
</code>

And we have a goal state:

<code>
goal = [
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)"
]
</code>

The actions/operators are:

<code>
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}
</code>

These will all need to be parsed from s-expressions to the underlying Python representation before you can use them. You might as well do it at the start of your algorithm, once. The order of the conditions is *not* arbitrary. It is much, much better for the unification and backtracking if you have the "type" predicates (item, place, agent) before the more complex ones. Trust me on this.

As for the algorithm itself, there is going to be an *outer* level of search and an *inner* level of search.

The *outer* level of search that is exactly what I describe here: you have a state, you generate successor states by applying actions to the current state, you examine those successor states as we did at the first week of the semester and if one is the goal you stop, if you see a repeat state, you put it on the explored list (you should implement graph search not tree search). What could be simpler?

It turns out the Devil is in the details. There is an *inner* level of search hidden in "you generate successor states by applying actions to the current state". Where?

How do you know if an action applies in a state? Only if the preconditions successfully unify with the current state. That seems easy enough...you check each predicate in the conditions to see if it unifies with the current state and if it does, you use the substitution list on the action, the add and delete lists and create the successor state based on them.

Except for one small problem...there may be more than one way to unify an action with the current state. You must essentially search for all successful unifications of the candidate action and the current state. This is where my question through the semester applies, "how would you modify state space search to return all the paths to the goal?"

Unification can be seen as state space search by trying to unify the first precondition with the current state, progressively working your way through the precondition list. If you fail at any point, you may need to backtrack because there might have been another unification of that predicate that would succeed. Similarly, as already mentioned, there may be more than one.

So...by using unification and a properly defined <code>successors</code> function, you should be able to apply graph based search to the problem and return a "path" through the states from the initial state to the goal. You'll definitely want to use graph-based search since <code>( drive Me Store), (drive Me Home), (drive Me Store), (drive Me Home), (drive Me Store), (buy Me Store Drill), (drive Me Home)</code> is a valid plan.

Your function should return the plan...but if you pass an extra debug=True parameter, it should also return the intermediate *states* as well as the actions.

-----

&nbsp;

**Func**

Desc

In [3]:
def parse_s_expression_list(state):
    new_state = []
    for s_expression in state:
        new_state.append(parse(s_expression))
    return new_state

&nbsp;

**Func**

Desc

In [4]:
def parse_s_expressions(start_state, goal, actions):
    start_state = parse_s_expression_list(start_state)
    goal = parse_s_expression_list(goal)

    new_actions = {}
    for action in actions:
        new_actions[action] = {}
        for key, value in actions[action].iteritems():
            if type(value) == str:
                new_actions[action][key] = parse(value)
            elif type(value) == list:
                new_actions[action][key] = parse_s_expression_list(value)

    return start_state, goal, new_actions

&nbsp;

**Func**

Desc

In [5]:
# Check if a precondition unifies with a state
def get_unifications(precondition, state):
    unifications = []
    for s_expression in state:
        substitutions = unification(precondition, s_expression)
        if substitutions:
            unifications.append(substitutions)

    return unifications  # each of these unifications is a branch in DFS


&nbsp;

**Func**

Desc

In [6]:
def apply_substitutions(preconditions, unification_dict):
    new_preconditions = copy.deepcopy(preconditions)
    for unified_variable in unification_dict:
        for i in range(len(new_preconditions)):
            for j in range(len(new_preconditions[i])):
                if new_preconditions[i][j] == unified_variable:
                    new_preconditions[i][j] = unification_dict[unified_variable]

    return new_preconditions

&nbsp;

**Func**

Desc

In [7]:
def find_next_precondition(preconditions):
    for expression_list in preconditions:
        for item in expression_list:
            if is_variable(item):
                return expression_list

    return None

&nbsp;

**Func**

Desc

In [8]:
# Inner DFS
def find_all_unifications(state, action, actions):
    all_unifications = []
    preconditions = copy.deepcopy(actions[action]['conditions'])
    start_unifications = get_unifications(preconditions[0], state)[0]
    start_preconditions = apply_substitutions(preconditions, start_unifications)
    stack = [(start_unifications, start_preconditions)]
    visited = [start_unifications]

    while stack:
        (current_unifications, current_preconditions) = stack.pop()

        if current_unifications not in all_unifications:
            all_unifications.append(current_unifications)

        next_precondition_to_unify = find_next_precondition(current_preconditions)

        if next_precondition_to_unify:
            unifications = get_unifications(next_precondition_to_unify, state)

            for unification in unifications:
                if unification not in visited:
                    new_preconditions = apply_substitutions(current_preconditions, unification)
                    stack.append((unification, new_preconditions))
                    visited.append(unification)

    return all_unifications

&nbsp;

**Func**

Desc

In [9]:
def get_separate_unifications(state, action, actions, all_unifications):
    combinations = {}
    for unification_dict in all_unifications:
        keys = unification_dict.keys()

        for key in keys:
            if key in combinations:
                combinations[key].append(unification_dict[key])
            else:
                combinations[key] = [unification_dict[key]]

    if action == 'drive':
        # SWITCH ordering of from, to to mix it up
        if combinations['?from'] == combinations['?to']:
            temp = combinations['?from'][0]
            combinations['?from'][0] = combinations['?from'][1]
            combinations['?from'][1] = temp

    unification1 = {}
    unification2 = {}

    for key in combinations:
        if len(combinations[key]) > 1:
            unification1[key] = combinations[key][0]
            unification2[key] = combinations[key][1]
        elif len(combinations[key]) == 1:
            unification1[key] = combinations[key][0]
            unification2[key] = combinations[key][0]

    return [unification1, unification2]

&nbsp;

**Func**

Desc

In [10]:
def apply_unifications(state, action, actions, unification_dict):
    adds = copy.deepcopy(actions[action]['add'][0])
    deletes = copy.deepcopy(actions[action]['delete'][0])
    new_state = copy.deepcopy(state)
    action = copy.deepcopy(actions[action]['action'])

    for unified_variable in unification_dict:  # TODO make sure these substitutions are actually being made
        for i in range(len(adds)):
            if adds[i] == unified_variable:
                adds[i] = unification_dict[unified_variable]

        for i in range(len(deletes)):
            if deletes[i] == unified_variable:
                deletes[i] = unification_dict[unified_variable]

        for i in range(len(action)):
            if action[i] == unified_variable:
                action[i] = unification_dict[unified_variable]

    new_state.append(adds)

    if deletes in new_state:
        new_state.remove(deletes)
    else:
        return None, None

    return new_state, action


&nbsp;

**Func**

Desc

In [11]:
# Generate successor states by applying actions to the current state
# There is an inner level of search in this generation
# We know if an action applies in a state IF the preconditions unify with the state
# Check each predicate in the conditions to see if it unifies with the state
# If it does, use the substitution list on the action, the add and delete lists and create the successor state based on them
# There may be more than one way to unify an action with the current state
# Search for all successful unifications of the candidate action and the current state
# Unification can be seen as state space search by trying to unify the first precondition with the current state,
# progressively working your way through the precondition list
# If you fail at any point, you may need to backtrack - there might have been another unification of that predicate that would succeed
def get_successors(state, actions):
    successor_states = []
    actions_taken = []
    for action in actions:
        all_unifications = find_all_unifications(state, action, actions)
        separate_unifications = get_separate_unifications(state, action, actions, all_unifications)

        for unification_dict in separate_unifications:
            new_state, action_taken = apply_unifications(state, action, actions, unification_dict)
            if new_state and action_taken:
                actions_taken.append(action_taken)
                successor_states.append(new_state)

    return successor_states, actions_taken


&nbsp;

**Func**

Desc

In [12]:
def construct_path(goal, state_parent_pairs, debug):
    path = []
    for tup in state_parent_pairs:
        if debug:
            if tup[2]:
                path.append(tup[0])
                path.append(tup[2])
        else:
            if tup[2]:
                path.append(tup[2])
    return path

(you can just overwrite that one and add as many others as you need). Remember to follow the **Guidelines**.


-----

So you need to implement `forward_planner` as described above. `start_state`, `goal` and `actions` should all have the layout above and be s-expressions.

Your implementation should return the plan as a **List of instantiated actions**. If `debug=True`, you should print out the intermediate states of the plan as well.

In [13]:
# So you need to implement `forward_planner` as described above. `start_state`, `goal` and `actions` should all have the layout above and be s-expressions.
# Return the plan as a **List of instantiated actions**. If `debug=True`, you should print out the intermediate states of the plan as well.
def forward_planner(start_state, goal, actions, debug=False):
    start_state, goal, actions = parse_s_expressions(start_state, goal, actions)
    frontier = [start_state]
    explored = []
    state_parent_tuples = [(start_state, None, None)]

    while frontier:
        current_state = frontier.pop(0)

        if current_state == goal:
            return construct_path(current_state, state_parent_tuples, debug)

        children, actions_taken = get_successors(current_state, actions)

        for i in range(len(children)):
            if children[i] not in explored and children[i] not in frontier:
                state_parent_tuples.append((children[i], current_state, actions_taken[i]))
                frontier.insert(0, children[i])

        explored.append(current_state)

    return []


You will be solving the problem from above. Here is the start state:

In [14]:
start_state = [
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Store)"
]

The goal state:

In [15]:
goal = [
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)"
]

and the actions/operators:

In [16]:
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}

In [17]:
plan = forward_planner( start_state, goal, actions)
pp.pprint(plan) 

[   ['buy', 'Me', 'Store', 'Drill'],
    ['drive', 'Me', 'Home', 'Store'],
    ['buy', 'Me', 'Store', 'Drill'],
    ['drive', 'Me', 'Store', 'Home'],
    ['drive', 'Me', 'Store', 'Home'],
    ['drive', 'Me', 'Home', 'Store']]


In [18]:
plan_with_states = forward_planner( start_state, goal, actions, debug=True)
pp.pprint(plan_with_states)

[   [   ['item', 'Drill'],
        ['place', 'Home'],
        ['place', 'Store'],
        ['agent', 'Me'],
        ['at', 'Me', 'Home'],
        ['at', 'Drill', 'Me']],
    ['buy', 'Me', 'Store', 'Drill'],
    [   ['item', 'Drill'],
        ['place', 'Home'],
        ['place', 'Store'],
        ['agent', 'Me'],
        ['at', 'Drill', 'Store'],
        ['at', 'Me', 'Store']],
    ['drive', 'Me', 'Home', 'Store'],
    [   ['item', 'Drill'],
        ['place', 'Home'],
        ['place', 'Store'],
        ['agent', 'Me'],
        ['at', 'Me', 'Store'],
        ['at', 'Drill', 'Me']],
    ['buy', 'Me', 'Store', 'Drill'],
    [   ['item', 'Drill'],
        ['place', 'Home'],
        ['place', 'Store'],
        ['agent', 'Me'],
        ['at', 'Drill', 'Store'],
        ['at', 'Me', 'Home']],
    ['drive', 'Me', 'Store', 'Home'],
    [   ['item', 'Drill'],
        ['place', 'Home'],
        ['place', 'Store'],
        ['agent', 'Me'],
        ['at', 'Drill', 'Me'],
        ['at', 'Me', 'Home']